# TP : Comment les LLM représentent le monde ?
## Exploration des embeddings et des biais géopolitiques

### Objectifs du TP
- Comprendre le concept d'embedding et de similarité sémantique
- Explorer comment les modèles de langage représentent les concepts géopolitiques
- Analyser les biais potentiels dans ces représentations
- Visualiser les relations sémantiques capturées par les embeddings

## 0. Installation et imports

In [ ]:
# Installation des librairies nécessaires
!pip install sentence-transformers numpy pandas matplotlib seaborn scikit-learn plotly -q

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px
import plotly.graph_objects as go
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

# Configuration pour un affichage optimal
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")

## 1. Introduction aux embeddings

### Qu'est-ce qu'un embedding ?

Un **embedding** est une représentation vectorielle dense d'un mot, d'une phrase ou d'un document. Ces vecteurs capturent les relations sémantiques entre les concepts.

### Pourquoi c'est important ?
- Les embeddings permettent de calculer des **similarités** entre concepts
- Ils révèlent comment le modèle "comprend" les relations entre les mots
- **MAIS** : ils reflètent aussi les biais présents dans les données d'entraînement !

## 2. Chargement du modèle

Nous utilisons un modèle multilingue pour pouvoir travailler avec différentes langues.

In [ ]:
# Chargement du modèle sentence-transformers multilingue
# Ce modèle est entraîné sur des données multilingues et peut encoder des phrases dans plus de 50 langues
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
model = SentenceTransformer(model_name)

print(f"Modèle chargé : {model_name}")
print(f"Dimension des embeddings : {model.get_sentence_embedding_dimension()}")

## 3. Fonctions utilitaires

Définissons les fonctions principales pour notre exploration.

In [ ]:
def generate_embeddings(texts: List[str], model: SentenceTransformer) -> Dict[str, np.ndarray]:
    """
    Génère les embeddings pour une liste de textes.
    
    Args:
        texts: Liste de textes à encoder
        model: Modèle SentenceTransformer
    
    Returns:
        Dictionnaire {texte: embedding}
    """
    embeddings = model.encode(texts, convert_to_numpy=True)
    return dict(zip(texts, embeddings))


def plot_similarity_matrix(embeddings_dict: Dict[str, np.ndarray], title: str = "Matrice de similarité"):
    """
    Affiche une matrice de similarité sous forme de heatmap.
    
    Args:
        embeddings_dict: Dictionnaire {texte: embedding}
        title: Titre du graphique
    """
    labels = list(embeddings_dict.keys())
    embeddings = np.array(list(embeddings_dict.values()))
    
    # Calcul de la matrice de similarité cosinus
    similarity_matrix = cosine_similarity(embeddings)
    
    # Création de la heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(similarity_matrix, 
                xticklabels=labels, 
                yticklabels=labels, 
                annot=True, 
                fmt='.2f',
                cmap='RdBu_r',
                vmin=-1, vmax=1,
                square=True)
    plt.title(title)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    return similarity_matrix


def plot_pca_2d(embeddings_dict: Dict[str, np.ndarray], title: str = "Projection PCA 2D", 
                categories: Dict[str, str] = None):
    """
    Projette les embeddings en 2D avec PCA et les visualise.
    
    Args:
        embeddings_dict: Dictionnaire {texte: embedding}
        title: Titre du graphique
        categories: Dictionnaire optionnel {texte: catégorie} pour colorer les points
    """
    labels = list(embeddings_dict.keys())
    embeddings = np.array(list(embeddings_dict.values()))
    
    # PCA en 2D
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(embeddings)
    
    # Création du DataFrame pour plotly
    df = pd.DataFrame({
        'x': embeddings_2d[:, 0],
        'y': embeddings_2d[:, 1],
        'label': labels
    })
    
    if categories:
        df['category'] = [categories.get(label, 'Autre') for label in labels]
        color_col = 'category'
    else:
        color_col = None
    
    # Visualisation interactive avec plotly
    fig = px.scatter(df, x='x', y='y', text='label', color=color_col,
                     title=title, 
                     labels={'x': f'PC1 ({pca.explained_variance_ratio_[0]:.1%})', 
                             'y': f'PC2 ({pca.explained_variance_ratio_[1]:.1%})'})
    
    fig.update_traces(textposition='top center', 
                      marker=dict(size=10))
    
    fig.update_layout(width=800, height=600)
    fig.show()
    
    print(f"Variance expliquée : PC1={pca.explained_variance_ratio_[0]:.1%}, PC2={pca.explained_variance_ratio_[1]:.1%}")
    print(f"Variance totale expliquée : {sum(pca.explained_variance_ratio_):.1%}")

## 4. Premier exemple : Concepts simples en français

Commençons par explorer comment le modèle représente des concepts simples de la langue française.

In [ ]:
# Exemple 1 : Mots simples
mots_simples = [
    "chat", "chien", "oiseau",  # Animaux
    "voiture", "avion", "train",  # Transports
    "pomme", "orange", "banane",  # Fruits
    "Paris", "Londres", "Berlin"  # Villes
]

# Génération des embeddings
embeddings_simples = generate_embeddings(mots_simples, model)

# Visualisation
print("=== Matrice de similarité ===\n")
similarity_matrix = plot_similarity_matrix(embeddings_simples, "Similarité entre concepts simples")

In [ ]:
# Définition des catégories pour la visualisation
categories_simples = {
    "chat": "Animaux", "chien": "Animaux", "oiseau": "Animaux",
    "voiture": "Transports", "avion": "Transports", "train": "Transports",
    "pomme": "Fruits", "orange": "Fruits", "banane": "Fruits",
    "Paris": "Villes", "Londres": "Villes", "Berlin": "Villes"
}

# Visualisation PCA
print("\n=== Projection PCA 2D ===\n")
plot_pca_2d(embeddings_simples, "Projection PCA - Concepts simples", categories_simples)

### 💡 Question 1
**Observez la matrice de similarité et la projection PCA. Que remarquez-vous ?**
- Les mots de la même catégorie sont-ils proches ?
- Y a-t-il des similarités surprenantes ?

## 5. Exploration géopolitique

Passons maintenant à l'exploration de concepts géopolitiques plus complexes.

### 5.1 Notions géopolitiques fondamentales

In [ ]:
# Concepts géopolitiques
concepts_geopolitiques = [
    "démocratie",
    "dictature",
    "liberté",
    "oppression",
    "développement",
    "pauvreté",
    "paix",
    "guerre",
    "occident",
    "orient",
    "nord global",
    "sud global"
]

embeddings_concepts = generate_embeddings(concepts_geopolitiques, model)
plot_similarity_matrix(embeddings_concepts, "Similarité entre concepts géopolitiques")

In [ ]:
# Catégorisation pour la visualisation
categories_concepts = {
    "démocratie": "Système politique",
    "dictature": "Système politique",
    "liberté": "Valeurs",
    "oppression": "Valeurs",
    "développement": "Économie",
    "pauvreté": "Économie",
    "paix": "État",
    "guerre": "État",
    "occident": "Géographie",
    "orient": "Géographie",
    "nord global": "Géographie",
    "sud global": "Géographie"
}

plot_pca_2d(embeddings_concepts, "Projection PCA - Concepts géopolitiques", categories_concepts)

### 💡 Question 2
**Analysez les associations entre concepts :**
- Quels concepts sont les plus proches de "démocratie" ?
- Comment sont positionnés "occident" et "orient" par rapport aux autres concepts ?
- Ces associations vous semblent-elles neutres ou biaisées ?

### 5.2 Représentation des pays

In [ ]:
# Liste de pays diversifiés
pays = [
    # Europe
    "France", "Allemagne", "Royaume-Uni", "Italie", "Espagne", "Pologne",
    # Amérique
    "États-Unis", "Canada", "Brésil", "Argentine", "Mexique",
    # Asie
    "Chine", "Japon", "Inde", "Corée du Sud", "Indonésie",
    # Afrique
    "Nigeria", "Afrique du Sud", "Égypte", "Kenya", "Maroc",
    # Moyen-Orient
    "Arabie Saoudite", "Iran", "Israël", "Turquie",
    # Océanie
    "Australie", "Nouvelle-Zélande"
]

embeddings_pays = generate_embeddings(pays, model)

In [ ]:
plot_similarity_matrix(embeddings_pays, "Matrice de similarité entre pays")

In [ ]:
# Définition des régions pour la visualisation
regions = {
    "France": "Europe", "Allemagne": "Europe", "Royaume-Uni": "Europe", 
    "Italie": "Europe", "Espagne": "Europe", "Pologne": "Europe",
    "États-Unis": "Amérique", "Canada": "Amérique", "Brésil": "Amérique", 
    "Argentine": "Amérique", "Mexique": "Amérique",
    "Chine": "Asie", "Japon": "Asie", "Inde": "Asie", 
    "Corée du Sud": "Asie", "Indonésie": "Asie",
    "Nigeria": "Afrique", "Afrique du Sud": "Afrique", "Égypte": "Afrique", 
    "Kenya": "Afrique", "Maroc": "Afrique",
    "Arabie Saoudite": "Moyen-Orient", "Iran": "Moyen-Orient", 
    "Israël": "Moyen-Orient", "Turquie": "Moyen-Orient",
    "Australie": "Océanie", "Nouvelle-Zélande": "Océanie"
}

plot_pca_2d(embeddings_pays, "Projection PCA - Pays du monde", regions)

### 💡 Question 3
**Explorez les regroupements de pays :**
- Les pays sont-ils regroupés par région géographique ?
- Ou par d'autres critères (économiques, culturels, politiques) ?
- Identifiez des pays "proches" de manière surprenante.

### 5.3 Pays dans différents contextes géopolitiques

In [ ]:
# Créons des descriptions contextualisées
contextes_pays = [
    # Contexte économique
    "France pays développé",
    "France économie avancée",
    "Nigeria pays en développement",
    "Nigeria économie émergente",
    
    # Contexte politique
    "États-Unis démocratie",
    "États-Unis superpuissance",
    "Chine régime autoritaire",
    "Chine puissance montante",
    
    # Contexte culturel
    "Japon tradition moderne",
    "Japon technologie avancée",
    "Inde diversité culturelle",
    "Inde démocratie peuplée",
    
    # Contexte conflictuel
    "Israël conflit régional",
    "Palestine territoire occupé",
    "Ukraine guerre 2022",
    "Russie agression militaire"
]

embeddings_contextes = generate_embeddings(contextes_pays, model)
plot_similarity_matrix(embeddings_contextes, "Similarité entre pays en contexte")

In [ ]:
# Catégorisons par type de contexte
categories_contextes = {
    "France pays développé": "Économique",
    "France économie avancée": "Économique",
    "Nigeria pays en développement": "Économique",
    "Nigeria économie émergente": "Économique",
    "États-Unis démocratie": "Politique",
    "États-Unis superpuissance": "Politique",
    "Chine régime autoritaire": "Politique",
    "Chine puissance montante": "Politique",
    "Japon tradition moderne": "Culturel",
    "Japon technologie avancée": "Culturel",
    "Inde diversité culturelle": "Culturel",
    "Inde démocratie peuplée": "Culturel",
    "Israël conflit régional": "Conflictuel",
    "Palestine territoire occupé": "Conflictuel",
    "Ukraine guerre 2022": "Conflictuel",
    "Russie agression militaire": "Conflictuel"
}

plot_pca_2d(embeddings_contextes, "Projection PCA - Pays en contexte", categories_contextes)

### 💡 Question 4
**Analysez l'impact du contexte :**
- Comment le contexte change-t-il la position d'un pays dans l'espace des embeddings ?
- Les descriptions positives/négatives influencent-elles les similarités ?
- Quels biais potentiels identifiez-vous ?

## 6. Exploration libre et réflexion

### 6.1 Testez vos propres hypothèses

In [ ]:
# Zone d'expérimentation libre
# Modifiez cette liste pour tester vos propres concepts !

mes_concepts = [
    # Exemple : stéréotypes de genre
    "homme fort",
    "femme forte",
    "homme sensible",
    "femme sensible",
    
    # Exemple : professions
    "ingénieur",
    "infirmière",
    "PDG",
    "secrétaire",
    
    # Ajoutez vos propres concepts ici !
    
]

# Génération et visualisation
embeddings_libres = generate_embeddings(mes_concepts, model)
plot_similarity_matrix(embeddings_libres, "Exploration libre - Mes concepts")
plot_pca_2d(embeddings_libres, "Projection PCA - Mes concepts")

### 6.2 Analyse comparative multilingue

In [ ]:
# Testons la représentation d'un même concept dans différentes langues
concepts_multilingues = [
    # Liberté
    "liberté",        # Français
    "freedom",        # Anglais
    "libertad",       # Espagnol
    "Freiheit",       # Allemand
    "自由",           # Chinois
    "حرية",          # Arabe
    
    # Paix
    "paix",          # Français
    "peace",         # Anglais
    "paz",           # Espagnol
    "Frieden",       # Allemand
    "和平",          # Chinois
    "سلام",          # Arabe
]

embeddings_multilingue = generate_embeddings(concepts_multilingues, model)
plot_similarity_matrix(embeddings_multilingue, "Similarité multilingue")

### 💡 Question 5
**Réflexion finale :**
- Les mêmes concepts sont-ils représentés de la même façon dans différentes langues ?
- Quelles implications cela a-t-il pour l'utilisation de modèles multilingues ?
- Comment pourrait-on détecter et corriger les biais identifiés ?

## 7. Pour aller plus loin

### 7.1 Comparaison avec d'autres modèles

In [ ]:
# Liste d'autres modèles à essayer
autres_modeles = [
    'sentence-transformers/all-MiniLM-L6-v2',  # Anglais uniquement
    'sentence-transformers/distiluse-base-multilingual-cased-v2',  # Multilingue
    'sentence-transformers/LaBSE',  # Large-scale multilingual
]

print("Autres modèles que vous pouvez tester :")
for modele in autres_modeles:
    print(f"- {modele}")

# Décommentez pour tester un autre modèle
# nouveau_model = SentenceTransformer('sentence-transformers/LaBSE')
# nouveaux_embeddings = generate_embeddings(pays[:10], nouveau_model)
# plot_pca_2d(nouveaux_embeddings, "Projection avec LaBSE")

## 8. Conclusions et discussions

### Points clés à retenir :

1. **Les embeddings capturent des relations sémantiques complexes**
   - Similarités conceptuelles
   - Relations géographiques et culturelles
   - Associations contextuelles

2. **Les biais sont omniprésents**
   - Reflet des données d'entraînement
   - Associations culturelles et historiques
   - Stéréotypes sociaux

3. **Implications pratiques**
   - Nécessité de tester et auditer les modèles
   - Importance du contexte d'utilisation
   - Besoin de diversité dans les données

### Questions pour la discussion :

- Comment peut-on utiliser ces modèles de manière éthique ?
- Quelles précautions prendre dans des applications sensibles ?
- Comment améliorer la représentation équitable des différentes cultures ?